In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.utils import Trainer, accuracy, infer_types
from neural_blueprints.config.architectures import TabularBERTConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.architectures import TabularBERT
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset
from neural_blueprints.preprocess import TabularPreprocessor

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)

2025-12-23 22:49:28,726 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-23 22:49:28,727 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,
    output_cardinalities = [3],
    latent_dim = 16,
    encoder_layers=8,
    dropout_p = 0.1,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

224
TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
  

TabularBERTConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], output_cardinalities=[3], latent_dim=16, encoder_layers=8, dropout_p=0.1, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [6]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='cross_entropy',
        optimizer='adam',
        early_stopping_patience=2,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=20, visualize=True)

2025-12-23 22:49:29,263 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:08<02:49,  8.94s/epoch]

Epoch 1/20, Training Loss: 0.9623, Validation Loss: 0.8993


Training Epochs:  10%|█         | 2/20 [00:17<02:40,  8.92s/epoch]

...

Training Epochs:  45%|████▌     | 9/20 [01:22<01:43,  9.38s/epoch]

Epoch 9/20, Training Loss: 0.8865, Validation Loss: 0.8936


Training Epochs:  50%|█████     | 10/20 [01:31<01:33,  9.32s/epoch]

Epoch 10/20, Training Loss: 0.8870, Validation Loss: 0.8889


2025-12-23 22:51:10,109 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 3 consecutive epochs. Early stopping at epoch 11.
Training Epochs:  50%|█████     | 10/20 [01:40<01:40, 10.08s/epoch]
2025-12-23 22:51:10,110 - neural_blueprints.utils.trainer - INFO - Training completed in 100.85 seconds.
2025-12-23 22:51:10,110 - neural_blueprints.utils.trainer - INFO - Best validation loss: 8.8395e-01


In [7]:
X = torch.tensor(val_dataset[:][0])
y = val_dataset[:][1]
with torch.no_grad():
    y_pred = model(X)
    y_pred = y_pred.argmax(dim=1)
print(f"Predictions: {y_pred[:5]}, \n Ground Truth: {y[:5]}")
acc = accuracy(y_pred, y)
print(f"Validation Accuracy: {acc:.4f}")

/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_7420/3930248022.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



Predictions: tensor([1, 1, 1, 1, 1]), 
 Ground Truth: tensor([2, 1, 2, 2, 2])
Validation Accuracy: 0.8573


### Masked Dataset Inference Accuracy

In [8]:
# Create dataset
dataset = MaskedTabularDataset(
    data, 
    discrete_features, 
    continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [9]:
bert_config = TabularBERTConfig(
    input_cardinalities = dataset.cardinalities,   
    latent_dim = 16,
    encoder_layers=8,
    dropout_p = 0.1,
    normalization = "batchnorm1d",
    activation = "gelu",
    final_activation = None
)

model = TabularBERT(bert_config)
model.blueprint()

TabularBERT(
  (input_projection): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
      

TabularBERTConfig(input_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], output_cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], latent_dim=16, encoder_layers=8, dropout_p=0.1, normalization='batchnorm1d', activation='gelu', final_activation=None)

In [10]:
trainer = Trainer(
    model=model,
    config=TrainerConfig(
        criterion='mixed_type_reconstruction_loss',
        optimizer='adam',
        early_stopping_patience=2,
        learning_rate=1e-3,
        weight_decay=1e-5,
        save_weights_path="./models/bert_adult.pth",
        batch_size=128
    )
)
trainer.train(train_dataset, val_dataset, epochs=20, visualize=True)

2025-12-23 22:51:10,524 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:24<07:49, 24.69s/epoch]

Epoch 1/20, Training Loss: 177.1215, Validation Loss: 164.4997


Training Epochs:  10%|█         | 2/20 [00:49<07:22, 24.61s/epoch]

...

Training Epochs:  50%|█████     | 10/20 [04:06<04:05, 24.59s/epoch]

Epoch 10/20, Training Loss: 162.1748, Validation Loss: 161.4327


Training Epochs:  55%|█████▌    | 11/20 [04:31<03:42, 24.72s/epoch]

Epoch 11/20, Training Loss: 162.1409, Validation Loss: 161.4787


2025-12-23 22:56:06,060 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 2 consecutive epochs. Early stopping at epoch 12.
Training Epochs:  55%|█████▌    | 11/20 [04:55<04:01, 26.87s/epoch]
2025-12-23 22:56:06,060 - neural_blueprints.utils.trainer - INFO - Training completed in 295.53 seconds.
2025-12-23 22:56:06,061 - neural_blueprints.utils.trainer - INFO - Best validation loss: 1.6143e+02


In [11]:
X = val_dataset[:][0]
y = val_dataset[:][1]
mask = val_dataset[:][2]
with torch.no_grad():
    y_pred = model(x=X)

dis_accuracy = 0
cont_accuracy = 0
for column_idx, column_name in enumerate(data.columns):
    print(f"\nFeature Column {column_name}:")
    predicted_attributes = y_pred[column_idx]      # shape: (batch_size, num_classes)
    targets = y[:, column_idx]                     # shape: (batch_size,)

    feature_mask = mask[:, column_idx]                  # shape: (batch_size,)
    predicted_attributes = predicted_attributes[feature_mask]
    if predicted_attributes.size(1) > 1:
        predicted_attributes = predicted_attributes.softmax(dim=-1).argmax(dim=-1).cpu().numpy()
    else:
        predicted_attributes = predicted_attributes.squeeze(-1).cpu().numpy()
    targets = targets[feature_mask].cpu().numpy()

    print("Predicted attribute values:", predicted_attributes[:5])
    print("True attribute values:", targets[:5])

    accuracy_value = accuracy(torch.tensor(predicted_attributes), torch.tensor(targets))
    print(f"Accuracy: {accuracy_value:.4f}")
    if column_name in discrete_features:
        dis_accuracy += accuracy_value
    else:
        cont_accuracy += accuracy_value

avg_dis_accuracy = dis_accuracy / len(discrete_features) if len(discrete_features) > 0 else 0
avg_cont_accuracy = cont_accuracy / len(continuous_features) if len(continuous_features) > 0 else 0
print(f"\nAverage Discrete Accuracy: {avg_dis_accuracy:.4f}")
print(f"Average Continuous Accuracy: {avg_cont_accuracy:.4f}")
avg_accuracy = (dis_accuracy + cont_accuracy) / len(data.columns)
print(f"Overall Average Accuracy: {avg_accuracy:.4f}")


Feature Column age:
Predicted attribute values: [0.15858339 0.3878646  0.27786005 0.36468259 0.38129112]
True attribute values: [0.01369863 0.28767124 0.23287672 0.38356164 0.4520548 ]
Accuracy: 0.2549

Feature Column workclass:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [1. 2. 4. 0. 4.]
Accuracy: 0.6856

Feature Column fnlwgt:
Predicted attribute values: [0.12759791 0.14422683 0.12402294 0.14477433 0.13359226]
True attribute values: [0.19283818 0.07218518 0.04431793 0.0255129  0.14118929]
Accuracy: 0.5354

Feature Column education:
Predicted attribute values: [12 12 12 10 12]
True attribute values: [12. 12. 12. 15. 12.]
Accuracy: 0.5725

Feature Column education-num:
Predicted attribute values: [ 2 16 16  2 16]
True attribute values: [ 2. 16. 16. 15. 16.]
Accuracy: 0.6009

Feature Column marital-status:
Predicted attribute values: [5 3 5 3 3]
True attribute values: [1. 3. 5. 3. 3.]
Accuracy: 0.7120

Feature Column occupation:
Predicted attribute values: [3 8 3 3 0